# Assignment 2 DSC 102 2022 FA

## Introduction

In this assignment we will conduct data engineering for the Amazon dataset. It is divided into 2 parts. The extracted features in Part 1 will be used for the Part 2 of assignment, where you train a model (or models) to predict user ratings for a product.

We will be using Apache Spark for this assignment. The default Spark API will be DataFrame, as it is now the recommended choice over the RDD API. That being said, please feel free to switch back to the RDD API if you see it as a better fit for the task. We provide you an option to request RDD format to start with. Also you can switch between DataFrame and RDD in your solution. 

Another newer API is Koalas, which is also avaliable. However, it has constraints and is not applicable to most tasks. Refer to the PA statement for detail.

### Set the following parameters

In [1]:
PID = 'a16195691' # your pid, for instance: 'a43223333'
INPUT_FORMAT = 'dataframe' # choose a format of your input data, valid options: 'dataframe', 'rdd', 'koalas'

In [2]:
# Boiler plates, do NOT modify
%load_ext autoreload
%autoreload 2
import os
import getpass
from pyspark.sql import SparkSession
from utilities import SEED
from utilities import PA2Test
from utilities import PA2Data
from utilities import data_cat
from pa2_main import PA2Executor
import time
if INPUT_FORMAT == 'dataframe':
    import pyspark.ml as M
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
if INPUT_FORMAT == 'koalas':
    import databricks.koalas as ks
elif INPUT_FORMAT == 'rdd':
    import pyspark.mllib as M
    from pyspark.mllib.feature import Word2Vec
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.linalg.distributed import RowMatrix

os.environ['PYSPARK_SUBMIT_ARGS'] = '--py-files utilities.py,assignment2.py \
--deploy-mode client \
pyspark-shell'

class args:
    review_filename = data_cat.review_filename
    product_filename = data_cat.product_filename
    product_processed_filename = data_cat.product_processed_filename
    ml_features_train_filename = data_cat.ml_features_train_filename
    ml_features_test_filename = data_cat.ml_features_test_filename
    output_root = '/home/{}/{}-pa2/test_results'.format(getpass.getuser(), PID)
    test_results_root = data_cat.test_results_root
    pid = PID

pa2 = PA2Executor(args, input_format=INPUT_FORMAT)
data_io = pa2.data_io
data_dict = pa2.data_dict
begin = time.time()


23/03/12 03:57:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/opt/bitnami/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Loading datasets ...Done


In [95]:
# Import your own dependencies



#-----------------------------

# Part 1: Feature Engineering

In [4]:
# Bring the part_1 datasets to memory and de-cache part_2 datasets. 
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_1')

# Task0: warm up 
This task is provided for you to get familiar with Spark API. We will use the dataframe API to demonstrate. Solution is given to you and this task won't be graded.

Refer to https://spark.apache.org/docs/latest/api/python/pyspark.sql.html for API guide.

The task is to implement the function below. Given the ```product_data``` table:
1. Take and print five rows.

1. Select only the ```asin``` column, then print five rows of it.

1. Select the row where ```asin = B00I8KEOTM``` and print it.

1. Count the total number of rows.

1. Calculate the mean ```price```.

1. You need to conduct the above operations, then extract some statistics out of the generated columns. You need to put the statistics in a python dictionary named ```res```. The description and schema of it are as follows:
    ```
    res
     | -- count_total: int -- count of total rows of the entire table after your operations
     | -- mean_price: float -- mean value of column price
    ```

In [5]:
data_dict["product"].show(5)

+----------+--------------------+--------------------+--------------------+-----+--------------------+
|      asin|           salesRank|          categories|               title|price|             related|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
|B00I8HVV6E|{Home &amp; Kitch...|[[Home & Kitchen,...|Intelligent Desig...|27.99|{also_viewed -> [...|
|B00I8KEOTM|                null|[[Apps for Androi...|                null| null|{also_viewed -> [...|
|B00I8KCW4G|{Clothing -> 2233...|[[Clothing, Shoes...|eShakti Women's P...|41.95|{also_viewed -> [...|
|B00I8JKCQW|{Clothing -> 1405...|[[Clothing, Shoes...|Lady Slimming Mid...| null|{also_viewed -> [...|
|B00I8JKI8E|{Home &amp; Kitch...|[[Clothing, Shoes...|3 Tier Bangle Bra...|24.99|{also_viewed -> [...|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



In [6]:
def task_0(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    product_data.show(5)
    product_data[['asin']].show(5)
    product_data.where(F.col('asin') == 'B00I8KEOTM').show()
    count_rows = product_data.count()
    mean_price = product_data.select(F.avg(F.col('price'))).head()[0]
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmatically. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {'count_total': None, 'mean_price': None}
    
    # Modify res:

    res['count_total'] = count_rows
    res['mean_price'] = mean_price

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    return res
    # -------------------------------------------------------------------------

In [7]:
if INPUT_FORMAT == 'dataframe':
    res = task_0(data_io, data_dict['product'])
    pa2.tests.test(res, 'task_0')

+----------+--------------------+--------------------+--------------------+-----+--------------------+
|      asin|           salesRank|          categories|               title|price|             related|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
|B00I8HVV6E|{Home &amp; Kitch...|[[Home & Kitchen,...|Intelligent Desig...|27.99|{also_viewed -> [...|
|B00I8KEOTM|                null|[[Apps for Androi...|                null| null|{also_viewed -> [...|
|B00I8KCW4G|{Clothing -> 2233...|[[Clothing, Shoes...|eShakti Women's P...|41.95|{also_viewed -> [...|
|B00I8JKCQW|{Clothing -> 1405...|[[Clothing, Shoes...|Lady Slimming Mid...| null|{also_viewed -> [...|
|B00I8JKI8E|{Home &amp; Kitch...|[[Clothing, Shoes...|3 Tier Bangle Bra...|24.99|{also_viewed -> [...|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows

+----------+
|      asin|
+----------+
|B00I8HVV

tests for task_0 --------------------------------------------------------------
2/2 passed
-------------------------------------------------------------------------------


In [8]:
# if INPUT_FORMAT == 'dataframe':
#     res = task_0(data_io, data_dict['product'])
#     print(res)

# Task1

In [9]:
data_dict["review"].printSchema()
data_dict["product"].printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- overall: float (nullable = true)

root
 |-- asin: string (nullable = true)
 |-- salesRank: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- price: float (nullable = true)
 |-- related: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: string (containsNull = true)



In [33]:
# %load -s task_1 assignment2.py
def task_1(data_io, review_data, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    groups = review_data.groupby("asin")
    
    newRev = groups.agg(
        F.avg("overall").alias("meanRating"),
        F.count("overall").alias("countRating")
    )

    
    newPrd = product_data.join(newRev, "asin", "left").select("asin", "meanRating", "countRating")
    
    vals = newPrd.select(
                        F.count("asin").alias("count_total"),
                        F.mean("meanRating").alias("mean_meanRating"), 
                        F.mean("countRating").alias("mean_countRating"),
                        F.variance("meanRating").alias("variance_meanRating"), 
                        F.variance("countRating").alias("variance_countRating"),
                        F.count(F.when(F.col("meanRating").isNull() , "meanRating")).alias("numNulls_meanRating"),
                        F.count(F.when(F.col("countRating").isNull() , "countRating")).alias("numNulls_countRating"),
                       ).collect()[0]
    
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmaticly. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {
        'count_total': vals["count_total"],
        'mean_meanRating': vals["mean_meanRating"],
        'variance_meanRating': vals["variance_meanRating"],
        'numNulls_meanRating': vals["numNulls_meanRating"],
        'mean_countRating': vals["mean_countRating"],
        'variance_countRating': vals["variance_countRating"],
        'numNulls_countRating': vals["numNulls_countRating"]
    }


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_1')
    return res
    # -------------------------------------------------------------------------

In [34]:
res = task_1(data_io, data_dict['review'], data_dict['product'])
pa2.tests.test(res, 'task_1')

tests for task_1 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countRating ... Pass
Test 3/7 : mean_meanRating ... Pass
Test 4/7 : numNulls_countRating ... Pass
Test 5/7 : numNulls_meanRating ... Pass
Test 6/7 : variance_countRating ... Pass
Test 7/7 : variance_meanRating ... Pass
7/7 passed
-------------------------------------------------------------------------------


True


# Task 2

In [169]:
# %load -s task_2 assignment2.py
def task_2(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    salesRank_column = 'salesRank'
    categories_column = 'categories'
    asin_column = 'asin'
    # Outputs:
    category_column = 'category'
    bestSalesCategory_column = 'bestSalesCategory'
    bestSalesRank_column = 'bestSalesRank'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    
    flat_data = product_data.withColumn(
            "category", 
            F.when(
                F.col("categories").getItem(0).getItem(0) != '', 
                F.col("categories").getItem(0).getItem(0)
            ).otherwise(None)
        ).withColumn(
            "bestSalesCategory", 
            F.map_keys(product_data["salesRank"]).getItem(0)
        ).withColumn(
            "bestSalesRank", 
            F.map_values(product_data["salesRank"]).getItem(0)
        ).select('asin', 'category', 'salesRank', 'bestSalesCategory', 'bestSalesRank')
    

    vals = flat_data.select(
        F.count("asin").alias("count_total"),
        F.mean("bestSalesRank").alias("mean_bestSalesRank"), 
        F.variance("bestSalesRank").alias("variance_bestSalesRank"),
        F.count(F.when(F.col("category").isNull() , "category")).alias("numNulls_category"),
        F.count(F.when(F.col("bestSalesRank").isNull() , "bestSalesRank")).alias("numNulls_bestSalesCategory"),
        F.countDistinct("category").alias("countDistinct_category"),
        F.countDistinct("bestSalesCategory").alias("countDistinct_bestSalesCategory"),
    ).collect()[0]

    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': vals['count_total'],
        'mean_bestSalesRank': vals['mean_bestSalesRank'],
        'variance_bestSalesRank': vals['variance_bestSalesRank'],
        'numNulls_category': vals['numNulls_category'],
        'countDistinct_category': vals['countDistinct_category'],
        'numNulls_bestSalesCategory': vals['numNulls_bestSalesCategory'],
        'countDistinct_bestSalesCategory': vals['countDistinct_bestSalesCategory']
    }
    # Modify res:
    print(res)

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_2')
    return res
    # -------------------------------------------------------------------------

In [170]:
res = task_2(data_io, data_dict['product'])
pa2.tests.test(res, 'task_2')

{'count_total': 9430000, 'mean_bestSalesRank': 1102421.3114573301, 'variance_bestSalesRank': 3616424023407.041, 'numNulls_category': 270169, 'countDistinct_category': 82, 'numNulls_bestSalesCategory': 2480772, 'countDistinct_bestSalesCategory': 33}
tests for task_2 --------------------------------------------------------------
Test 1/7 : countDistinct_bestSalesCategory ... Pass
Test 2/7 : countDistinct_category ... Pass
Test 3/7 : count_total ... Pass
Test 4/7 : mean_bestSalesRank ... Pass
Test 5/7 : numNulls_bestSalesCategory ... Pass
Test 6/7 : numNulls_category ... Pass
Test 7/7 : variance_bestSalesRank ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 3





In [239]:
# %load -s task_3 assignment2.py
def task_3(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    price_column = 'price'
    attribute = 'also_viewed'
    related_column = 'related'
    # Outputs:
    meanPriceAlsoViewed_column = 'meanPriceAlsoViewed'
    countAlsoViewed_column = 'countAlsoViewed'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    
    also_viewed = product_data.select(
        product_data['asin'],
        product_data["related"].getItem('also_viewed').alias('also_viewed'),
        F.when(
            F.size(product_data["related"].getItem('also_viewed')) > 0,
            F.size(product_data["related"].getItem('also_viewed'))
        ).otherwise(None).alias('countAlsoViewed')
    )

    exploded_vals = also_viewed.withColumn(
        "exp_vals",
        F.explode(F.col('also_viewed'))
    ).select('asin', 'exp_vals')
#     exploded_vals.show(10)
    
    prices = product_data.select(F.col('asin').alias('pasin'), 'price').filter(F.col('price').isNotNull())
#     prices.show(10)

    exp_prices = prices.join(exploded_vals,prices.pasin == exploded_vals.exp_vals,'left')
#     exp_prices.show(10)
    
    merged_prices = exp_prices.groupby('asin').agg(
        F.mean('price').alias('meanPriceAlsoViewed')
    )
#     merged_prices.show(10)
    outcome = also_viewed.join(merged_prices, on = 'asin', how = 'left')
#     outcome.show(10)
    
    vals = outcome.select(
        F.count("asin").alias("count_total"),
        F.mean("meanPriceAlsoViewed").alias("mean_meanPriceAlsoViewed"),
        F.mean("countAlsoViewed").alias("mean_countAlsoViewed"),
        F.variance("meanPriceAlsoViewed").alias("variance_meanPriceAlsoViewed"),
        F.variance("countAlsoViewed").alias("variance_countAlsoViewed"),
        F.count(F.when(F.col("meanPriceAlsoViewed").isNull() , "meanPriceAlsoViewed")).alias("numNulls_meanPriceAlsoViewed"),
        F.count(F.when(F.col("countAlsoViewed").isNull() , "countAlsoViewed")).alias("numNulls_countAlsoViewed"),
    ).collect()[0]


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': vals['count_total'],
        'mean_meanPriceAlsoViewed': vals['mean_meanPriceAlsoViewed'],
        'variance_meanPriceAlsoViewed': vals['variance_meanPriceAlsoViewed'],
        'numNulls_meanPriceAlsoViewed': vals['numNulls_meanPriceAlsoViewed'],
        'mean_countAlsoViewed': vals['mean_countAlsoViewed'],
        'variance_countAlsoViewed': vals['variance_countAlsoViewed'],
        'numNulls_countAlsoViewed': vals['numNulls_countAlsoViewed']
    }
    # Modify res:
    print(res)



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_3')
    return res
    # -------------------------------------------------------------------------

In [240]:
res = task_3(data_io, data_dict['product'])
pa2.tests.test(res, 'task_3')

{'count_total': 9430000, 'mean_meanPriceAlsoViewed': 45.28501828302965, 'variance_meanPriceAlsoViewed': 5135.311790051096, 'numNulls_meanPriceAlsoViewed': 5835773, 'mean_countAlsoViewed': 31.561228904913023, 'variance_countAlsoViewed': 562.8028467039506, 'numNulls_countAlsoViewed': 5408566}


tests for task_3 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countAlsoViewed ... Pass
Test 3/7 : mean_meanPriceAlsoViewed ... Pass
Test 4/7 : numNulls_countAlsoViewed ... Pass
Test 5/7 : numNulls_meanPriceAlsoViewed ... Pass
Test 6/7 : variance_countAlsoViewed ... Pass
Test 7/7 : variance_meanPriceAlsoViewed ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 4

In [ ]:
# %load -s task_4 assignment2.py
def task_4(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    price_column = 'price'
    title_column = 'title'
    # Outputs:
    meanImputedPrice_column = 'meanImputedPrice'
    medianImputedPrice_column = 'medianImputedPrice'
    unknownImputedTitle_column = 'unknownImputedTitle'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------





    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanImputedPrice': None,
        'variance_meanImputedPrice': None,
        'numNulls_meanImputedPrice': None,
        'mean_medianImputedPrice': None,
        'variance_medianImputedPrice': None,
        'numNulls_medianImputedPrice': None,
        'numUnknowns_unknownImputedTitle': None
    }
    # Modify res:




    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_4')
    return res
    # -------------------------------------------------------------------------


In [ ]:
res = task_4(data_io, data_dict['product'])
pa2.tests.test(res, 'task_4')

# Task 5

In [ ]:
# %load -s task_5 assignment2.py
def task_5(data_io, product_processed_data, word_0, word_1, word_2):
    # -----------------------------Column names--------------------------------
    # Inputs:
    title_column = 'title'
    # Outputs:
    titleArray_column = 'titleArray'
    titleVector_column = 'titleVector'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------





    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'size_vocabulary': None,
        'word_0_synonyms': [(None, None), ],
        'word_1_synonyms': [(None, None), ],
        'word_2_synonyms': [(None, None), ]
    }
    # Modify res:
    res['count_total'] = product_processed_data_output.count()
    res['size_vocabulary'] = model.getVectors().count()
    for name, word in zip(
        ['word_0_synonyms', 'word_1_synonyms', 'word_2_synonyms'],
        [word_0, word_1, word_2]
    ):
        res[name] = model.findSynonymsArray(word, 10)
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_5')
    return res
    # -------------------------------------------------------------------------


In [ ]:
res = task_5(data_io, data_dict['product_processed'], 'piano', 'rice', 'laptop')
pa2.tests.test(res, 'task_5')

# Task 6

In [ ]:
# %load -s task_6 assignment2.py
def task_6(data_io, product_processed_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    category_column = 'category'
    # Outputs:
    categoryIndex_column = 'categoryIndex'
    categoryOneHot_column = 'categoryOneHot'
    categoryPCA_column = 'categoryPCA'
    # -------------------------------------------------------------------------    

    # ---------------------- Your implementation begins------------------------





    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'meanVector_categoryOneHot': [None, ],
        'meanVector_categoryPCA': [None, ]
    }
    # Modify res:




    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_6')
    return res
    # -------------------------------------------------------------------------


In [ ]:
res = task_6(data_io, data_dict['product_processed'])
pa2.tests.test(res, 'task_6')

In [ ]:
print ("End to end time: {}".format(time.time()-begin))

# Part 2: Model Selection

In [ ]:
# Bring the part_2 datasets to memory and de-cache part_1 datasets.
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_2')

# Task 7

In [ ]:
def task_7(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    
    
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None
    }
    # Modify res:


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_7')
    return res
    # -------------------------------------------------------------------------

In [ ]:
res = task_7(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_7')

# Task 8

In [ ]:
def task_8(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    
    
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None,
        'valid_rmse_depth_5': None,
        'valid_rmse_depth_7': None,
        'valid_rmse_depth_9': None,
        'valid_rmse_depth_12': None,
    }
    # Modify res:


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_8')
    return res
    # -------------------------------------------------------------------------

In [ ]:
res = task_8(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_8')

In [ ]:
print ("End to end time: {}".format(time.time()-begin))